In [31]:
import pandas as pd
import os
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from copy import deepcopy

In [32]:
st = pd.HDFStore(os.path.expanduser("/Users/nipunbatra/wiki-all.h5"))

In [33]:
metadata_df = pd.read_csv("/Users/nipunbatra/w/dataport-metadata.csv",index_col=0)

In [34]:
months=3
start='7-1-2014'
start_month=7
end = '9-30-2014'
end_month=10

In [35]:
sd_df = pd.read_csv("../data/input/san_diego_df.csv", index_col=0)
bo_df = pd.read_csv("../data/input/boulder_df.csv", index_col=0)

In [36]:
import json
sd_homes = json.load(open("../data/input/san_diego_homes.json",'r'))
bo_homes = json.load(open("../data/input/boulder_homes.json",'r'))

In [37]:
K_min, K_max = 1,6
F_min, F_max=1,8

#from all_functions import *
#from features import *

import sys

from sklearn.neighbors import KNeighborsRegressor
from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import LeaveOneOut

NUM_NEIGHBOUR_MAX = 6
F_MAX = 4

import json

feature_map = {'Monthly':['aggregate_%d' %month for month in range(start_month, end_month)],
              'Static':['area'],
              'Monthly+Static':np.hstack([['aggregate_%d' %month for month in range(start_month, end_month)],
                                        'area']).flatten().tolist()}



from sklearn.cross_validation import LeaveOneOut
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from collections import OrderedDict

In [38]:
test_dataset_df = bo_df
train_dataset_df = sd_df
test_all_homes = bo_homes
train_all_homes = sd_homes

In [46]:
def _find_accuracy(home, appliance, feature="Monthly"):
    if appliance =="hvac":
        start, stop=7,10
    else:
        start, stop=7, 10

    np.random.seed(42)
    appliance_df = train_dataset_df.ix[train_all_homes[appliance]]
    appliance_df_test = test_dataset_df.ix[test_all_homes[appliance]]
    
    #ADDING NEW
    appliance_df = appliance_df.ix[appliance_df[feature_map[feature]].dropna().index]
    
    
    test_homes = [home]
    train_homes = appliance_df[~appliance_df.index.isin([home])].index
    all_home_appliance = deepcopy(train_all_homes)
    all_home_appliance[appliance] = train_homes

    # Cross validation on inner loop to find best feature, K
    train_size = len(train_homes)
    l = LeaveOneOut(train_size)
    out = OrderedDict()
    for cv_train, cv_test in l:

        cv_train_home=appliance_df.ix[train_homes[cv_train]]
        cv_test_home = appliance_df.ix[train_homes[cv_test]]
        test_home_name = cv_test_home.index.values[0]
        #print cv_test_home
        out[test_home_name]={}


        # Summing up energy across start to stop to get Y to learn optimum feature on
        Y = cv_train_home[['%s_%d' %(appliance, i) for i in range(start, stop)]].sum(axis=1).values
        forest = ExtraTreesRegressor(n_estimators=250,
                              random_state=0)
        forest.fit(cv_train_home[feature_map[feature]], Y)
        importances = forest.feature_importances_
        indices = np.argsort(importances)[::-1]

        # Now varying K and top-N features

        for K in range(K_min, K_max):
            out[test_home_name][K]={}
            for top_n in range(F_min,F_max):
                out[test_home_name][K][top_n]=[]
                top_n_features = cv_train_home[feature_map[feature]].columns[indices][:top_n]

                # Now fitting KNN on this
                for month in range(start, stop):
                    clf = KNeighborsRegressor(n_neighbors=K)
                    clf.fit(cv_train_home[top_n_features], cv_train_home['%s_%d' %(appliance, month)])
                    out[test_home_name][K][top_n].append(clf.predict(cv_test_home[top_n_features]))

        # Now, finding the (K, top_n) combination that gave us best accuracy on CV test homes
    accur = {}

    for K in range(K_min, K_max):
        accur[K] = {}
        for top_n in range(F_min, F_max):
            temp = {}
            for h in out.iterkeys():
                pred = pd.DataFrame(out[h][K][top_n]).T
                #all_but_h = [x for x in out.keys() if x!=h]
                pred.index = [h]
                pred.columns = [['%s_%d' %(appliance, i) for i in range(start, stop)]]
                gt = appliance_df.ix[h][['%s_%d' %(appliance, i) for i in range(start, stop)]]
                error = (pred-gt).abs().div(gt).mul(100)
                mean_error = error.mean().mean()
                a = 100-mean_error
                if a<0:
                    a=0
                temp[h]=a
            ac = pd.Series(temp).mean()

            accur[K][top_n] = ac

    accur_df = pd.DataFrame(accur)
    accur_max = accur_df.max().max()
    max_ac_df = accur_df[accur_df==accur_max]
    F_best = cv_train_home[feature_map[feature]].columns[indices][:max_ac_df.mean(axis=1).dropna().index.values[0]].tolist()
    K_best = max_ac_df.mean().dropna().index.values[0]

    # Now predicting for test home
    
    train_overall = appliance_df.ix[appliance_df[~appliance_df.index.isin([home])].index]
    test_overall = appliance_df_test[appliance_df_test.index.isin([home])]
    pred_test = {}
    gt_test = {}
    for month in range(start, stop):
        clf = KNeighborsRegressor(n_neighbors=K_best)
        clf.fit(train_overall[F_best], train_overall['%s_%d' %(appliance, month)])
        pred_test[month] = clf.predict(test_overall[F_best])
        gt_test[month] = test_overall['%s_%d' %(appliance, month)]

    #print {'f':F_best, 'k':K_best,'accuracy':accur_max}
    #json.dump({'f':F_best, 'k':K_best,'accuracy':accur_max},open("../main-out-new/%s_%s_%d.json" %(appliance,feature, home),"w") )

    pred_df = pd.DataFrame(pred_test)
    pred_df.index = [home]
    gt_df = pd.DataFrame(gt_test)
    error = (gt_df-pred_df).abs().div(gt_df).mul(100)
    accuracy_test = 100-error
    accuracy_test[accuracy_test<0]=0

    return pred_df, gt_df, accuracy_test.squeeze()

In [47]:
acc = {}

In [48]:
#for appliance in ["hvac","fridge","dw","mw"]:
for appliance in ["hvac"]:
    if appliance in acc:
        continue
    else:
        print appliance
        acc[appliance] = {}
        feature="Monthly"
        for home in bo_homes[appliance][:]:
            try:
                pred, gt, out_df = _find_accuracy(home, appliance, feature)
                acc[appliance][home]=out_df
                print home, acc[appliance][home].mean()
            except Exeption, e:
                print e

hvac
232 56.9491058512
698 7.85623878312
1879 43.9140187427
2144 69.6734532103
2523 0.0
2903 37.4683061675
3734 49.8028277462
4147 15.0484026012
4920 29.2708887843
4974 34.3540318339
5652 19.3772452776
5904 48.2688098564
7001 43.3033481323
8273 11.6831637237
8368 48.3163073911
8395 43.6547024211
9156 45.826489418
9341 55.7079062651
9818 52.7298298878


In [52]:
pd.DataFrame(acc['hvac']).T.mean()

7    56.894190
8    49.585235
9     6.131903
dtype: float64

In [12]:
#for appliance in ["hvac","fridge","dw","mw"]:
for appliance in ["fridge","hvac"]:
    if appliance in acc:
        continue
    else:
        print appliance
        acc[appliance] = {}
        feature="Monthly"
        for home in bo_homes[appliance][:]:
            try:
                pred, gt, out_df = _find_accuracy(home, appliance, feature)
                acc[appliance][home]=out_df
                print home, acc[appliance][home].mean()
            except Exeption, e:
                print e

fridge
698 96.5397271845
1350 67.8035880339
1879 58.7388484772
2144 97.0705590542
2523 90.3224642219
2903 55.129893089
3104 75.5938205903
3353 41.6497319685
3734 70.7178716102
4601 86.5695956932
4703 60.0667680629
4830 83.6099476131
4920 63.6088584776
4974 63.3826177532
5652 84.7860542967
5904 60.0681232139
6148 57.9375172364
6264 27.893513418
6888 89.4637942679
6956 58.2080819012
7818 44.3389132385
8273 55.8005155956
8368 64.5161186165
8622 83.3254787276
8729 36.7405109653
8862 77.1903734155
9085 78.7771532917
9341 94.0508955593
9818 91.7869015655
hvac
232 91.8903077922
698 0.0
1879 23.9480543358
2144 67.6237720249
2523 0.0
2903 0.0
3734 56.225860573
4147 63.6444349687
4920 0.0
4974 87.5829012459
5652 0.0
5904 90.821901653
7001 52.6623072259
8273 4.02898525246
8368 81.7060763746
8395 50.1667861585
9156 60.7619690893
9341 91.9657155
9818 94.5015282017


In [42]:
pd.Series(acc['hvac']).mean()

46.353887346772474

In [43]:
mean_across_months = {}
for appliance in acc.keys():
    if appliance =="hvac":
        mean_across_months[appliance] = pd.Series(acc[appliance])
    else:
        mean_across_months[appliance] = pd.DataFrame(acc[appliance])
    


mean_across_months['fridge'].loc[10, 2144] = np.NaN
mean_across_months['fridge'].loc[7, 7818] = np.NaN#mean_across_months['hvac'].loc[9, 3938] = np.NaN

for appliance in mean_across_months.keys():
    if appliance!="hvac":
        mean_across_months[appliance] = mean_across_months[appliance].mean()
    

In [44]:
pd.DataFrame(acc['fridge']).mean().mean()

68.50575613455632

In [45]:
pd.DataFrame(mean_across_months).mean()

fridge    69.474575
hvac      46.353887
dtype: float64